In [3]:
from requests_html import HTMLSession, HTML
import requests
import json
import time
import pandas as pd
import datetime as dt
import re
from selenium import webdriver
import urllib.robotparser

In [4]:
#Kiểm tra dữ liệu vận động viên lấy về có hợp pháp hay không
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.basketball-reference.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.basketball-reference.com/leagues/NBA_2019_per_game.html')

True

In [5]:
#Kiểm tra dữ liệu lương lấy về có hợp pháp hay không
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://hoopshype.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://hoopshype.com/salaries/players/')

True

In [102]:
#Tạo 1 dataframe rỗng để chứa thông tin crawl về
df = pd.DataFrame([], columns = ['name', #Tên vận động viên
                                 'pos',  #Vị trí chơi
                                 'age',  #Tuổi
                                 'team', #Đội bóng
                                 'game', #Số trận chơi
                                 'gs',   #Số trận ra sân trong đội hình chính
                                 'mp',   #Trung bình số phút thi đấu mỗi trận
                                 'fg',   #Trung bình số lần ghi bàn mỗi trận
                                 'fga',  #Trung bình số lần dứt điểm mỗi trận
                                 'fgp',  #Tỉ lệ dứt điểm thành công
                                 '3p',   #Trung bình số lần ghi bàn 3 điểm mỗi trận
                                 '3pa',  #Trung bình số lần dứt điểm 3 điểm mỗi trận
                                 '3pp',  #Tỉ lệ dứt điểm 3 điểm thành công
                                 '2p',   #Trung bình số lần ghi bàn 2 điểm mỗi trận
                                 '2pa',  #Trung bình số lần dứt điểm 2 điểm mỗi trận
                                 '2pp',  #Tỉ lệ dứt điểm 2 điểm thành công
                                 'efgp', #Hiệu suất ghi bàn mỗi trận (quả 3đ sẽ có trọng số khác so với quả 2 điểm)
                                 'ft',   #Trung bình số lần ghi bàn phạt
                                 'fta',  #Trung bình số lần quăng banh phạt
                                 'ftp',  #Tỷ lệ quăng banh phạt thành công
                                 'orb',  #Trung bình số lần bắt bóng (rebound) khi tấn công
                                 'drb',  #Trung bình số lần bắt bóng (rebound) phòng thủ
                                 'trb',  #Trung bình số lần bắt bóng (rebound)
                                 'ast',  #Trung bình số lần kiến tạo 
                                 'stl',  #Trung bình số lần cướp bóng mỗi trận
                                 'blk',  #Trung bình số lần chặn bóng mỗi trận
                                 'tov',  #Trung bình số lần mất bóng
                                 'pf',   #Trung bình số lỗi
                                 'pts',  #Trung bình số điểm ghi được
                                ])

In [103]:
#Set up mọi thứ để crawl
browser = webdriver.Chrome(executable_path='chromedriver.exe')
browser.get('https://www.basketball-reference.com/leagues/NBA_2019_per_game.html')
time.sleep(10)
html_text = HTML(html=browser.page_source)
time.sleep(3)
table = html_text.find('tbody')
row = table[25].find('tr')

In [104]:
#Crawl data vận động viên bóng rổ
for i in range(len(row)):
    print(i)
    s = row[i].find('td') 
    try:
        df.loc[len(df)] = [k.text for k in s]
    except:
        pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [105]:
df.head(10)

,name,pos,age,team,game,gs,mp,fg,fga,fgp,...,ftp,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,.357,...,.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,.222,...,.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,.345,...,.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,.595,...,.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,.576,...,.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
5,Deng Adel,SF,21,CLE,19,3,10.2,0.6,1.9,.306,...,1.000,0.2,0.8,1.0,0.3,0.1,0.2,0.3,0.7,1.7
6,DeVaughn Akoon-Purcell,SG,25,DEN,7,0,3.1,0.4,1.4,.300,...,.500,0.1,0.4,0.6,0.9,0.3,0.0,0.3,0.6,1.0
7,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,.519,...,.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
8,Rawle Alkins,SG,21,CHI,10,1,12.0,1.3,3.9,.333,...,.667,1.1,1.5,2.6,1.3,0.1,0.0,0.8,0.7,3.7
9,Grayson Allen,SG,23,UTA,38,2,10.9,1.8,4.7,.376,...,.750,0.1,0.5,0.6,0.7,0.2,0.2,0.9,1.2,5.6


In [124]:
#Thay thế những ô trống bằng số 0
df = df.replace('', '0')

In [125]:
#Chuyển type của các field
df = df.astype({
                 'age': 'int64',  #Tuổi
                 'game': 'int64', #Số trận chơi
                 'gs': 'int64',   #Số trận ra sân trong đội hình chính
                 'mp': 'float64',   #Trung bình số phút thi đấu mỗi trận
                 'fg': 'float64',   #Trung bình số lần ghi bàn mỗi trận
                 'fga': 'float64',  #Trung bình số lần dứt điểm mỗi trận
                 'fgp': 'float64',  #Tỉ lệ dứt điểm thành công
                 '3p': 'float64',   #Trung bình số lần ghi bàn 3 điểm mỗi trận
                 '3pa': 'float64',  #Trung bình số lần dứt điểm 3 điểm mỗi trận
                 '3pp': 'float64',  #Tỉ lệ dứt điểm 3 điểm thành công
                 '2p': 'float64',   #Trung bình số lần ghi bàn 2 điểm mỗi trận
                 '2pa': 'float64',  #Trung bình số lần dứt điểm 2 điểm mỗi trận
                 '2pp': 'float64',  #Tỉ lệ dứt điểm 2 điểm thành công
                 'efgp': 'float64', #Hiệu suất ghi bàn mỗi trận (quả 3đ sẽ có trọng số khác so với quả 2 điểm)
                 'ft': 'float64',   #Trung bình số lần ghi bàn phạt
                 'fta': 'float64',  #Trung bình số lần quăng banh phạt
                 'ftp': 'float64',  #Tỷ lệ quăng banh phạt thành công
                 'orb': 'float64',  #Trung bình số lần bắt bóng (rebound) khi tấn công
                 'drb': 'float64',  #Trung bình số lần bắt bóng (rebound) phòng thủ
                 'trb': 'float64',  #Trung bình số lần bắt bóng (rebound)
                 'ast': 'float64',  #Trung bình số lần kiến tạo 
                 'stl': 'float64',  #Trung bình số lần cướp bóng mỗi trận
                 'blk': 'float64',  #Trung bình số lần chặn bóng mỗi trận
                 'tov': 'float64',  #Trung bình số lần mất bóng
                 'pf': 'float64',   #Trung bình số lỗi
                 'pts': 'float64'   #Trung bình số điểm ghi được
})

In [126]:
#Tóm tắt cho thấy ta không có dữ liệu thiếu
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 708 entries, 0 to 707
Data columns (total 29 columns):
name    708 non-null object
pos     708 non-null object
age     708 non-null int64
team    708 non-null object
game    708 non-null int64
gs      708 non-null int64
mp      708 non-null float64
fg      708 non-null float64
fga     708 non-null float64
fgp     708 non-null float64
3p      708 non-null float64
3pa     708 non-null float64
3pp     708 non-null float64
2p      708 non-null float64
2pa     708 non-null float64
2pp     708 non-null float64
efgp    708 non-null float64
ft      708 non-null float64
fta     708 non-null float64
ftp     708 non-null float64
orb     708 non-null float64
drb     708 non-null float64
trb     708 non-null float64
ast     708 non-null float64
stl     708 non-null float64
blk     708 non-null float64
tov     708 non-null float64
pf      708 non-null float64
pts     708 non-null float64
dtypes: float64(23), int64(3), object(3)
memory usage: 165.9+ K

# Trong data có những tên cầu thủ bị trùng lặp do cầu thủ đó chuyển từ đội này sang đội khác nên ta sẽ tiến hành gộp những dòng đó lại thành 1 

In [144]:
lst = df.loc[df['team'] == 'TOT'].index
for i in lst:
    df.loc[i, 'team'] = df.loc[i+2, 'team']
    df = df.drop(i+1)
    df = df.drop(i+2)

In [145]:
df = df.reset_index(drop=True)

In [150]:
#Lưu dữ liệu
df.to_csv('player.csv', sep = '\t', index=False)

In [151]:
#Mở dữ liệu
df = pd.read_csv('player.csv', sep = '\t')
df.head()

,name,pos,age,team,game,gs,mp,fg,fga,fgp,...,ftp,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


### Bây giờ ta bắt đầu đi crawl tiền lương / năm của vận động viên

In [155]:
browser = webdriver.Chrome(executable_path='chromedriver.exe')
browser.get('https://hoopshype.com/salaries/players/')
time.sleep(10)
html_text = HTML(html=browser.page_source)
time.sleep(3)

In [198]:
#Crawl về dữ liệu lương của vận động viên
salary = pd.DataFrame([], columns = ['name', 'salary'])
name = html_text.find('.name')
s = html_text.find('.hh-salaries-sorted')
for i in range(len(name)): 
    sa = s[i].text[1:].replace(',', '')
    salary.loc[len(salary)] = [name[i].text, sa]

In [199]:
salary = salary.loc[salary['salary'] != '019/20']
salary = salary.reset_index(drop=True)

In [202]:
salary['salary'] = salary['salary'].astype('float64')/1000

In [203]:
salary.head()

,name,salary
0,Stephen Curry,40231.758
1,Russell Westbrook,38506.482
2,Chris Paul,38506.482
3,James Harden,38199.000
4,John Wall,38199.000


In [204]:
#Lưu dữ liệu lương
salary.to_csv('salary.csv', sep = '\t', index=False)

In [205]:
#Mở dữ liệu
salary = pd.read_csv('salary.csv', sep = '\t')
salary.head()

,name,salary
0,Stephen Curry,40231.758
1,Russell Westbrook,38506.482
2,Chris Paul,38506.482
3,James Harden,38199.000
4,John Wall,38199.000


### Ghép lương theo năm vào bảng thông tin tuyển thủ

In [6]:
import difflib

In [7]:
df = pd.read_csv('player.csv', sep = '\t')
df.head()

,name,pos,age,team,game,gs,mp,fg,fga,fgp,...,ftp,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


In [8]:
salary = pd.read_csv('salary.csv', sep = '\t')
salary.head()

,name,salary
0,Stephen Curry,40231.758
1,Russell Westbrook,38506.482
2,Chris Paul,38506.482
3,James Harden,38199.000
4,John Wall,38199.000


In [16]:
for i in range(len(salary)):
    try:
        salary['name'][i] = difflib.get_close_matches(salary['name'][i], df['name'])[0]
    except:
        pass

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
#Nối bảng lương vào dataframe
for i in range(len(salary)):
    df.loc[df['name'] == salary['name'][i], 'salary'] = salary['salary'][i]

In [23]:
#Xóa những cầu thủ không có lương công khai
df = df.dropna()

In [24]:
df.to_csv('player.csv', sep = '\t', index=False)